In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.autograd import Variable

## 1. Neural Network Architecture

### Load Data

In [2]:
mnist_train = datasets.MNIST(root='data/', 
                             train=True,
                             transform=transforms.ToTensor(),
                             download=True)

mnist_test = datasets.MNIST(root='data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

In [3]:
mnist_train.train_data.size(), mnist_test.test_data.size()

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [4]:
batch_size = 100

data_loader = data.DataLoader(dataset = mnist_train,
                              batch_size = batch_size,
                              shuffle = True,
                              num_workers = 0)

### Define Model

In [5]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        
        self.fc_layer = nn.Sequential(
            nn.Linear(784, 512, bias = True),
            nn.ReLU(),
            nn.Linear(512, 256, bias = True),
            nn.ReLU(),
            nn.Linear(256, 10, bias = True)
        )
        
    def forward(self, x):
        out = self.fc_layer(x)

        return out
    
model = DNN() #.cuda() 

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training

In [6]:
num_epochs = 10

for epoch in range(num_epochs):
    
    total_batch = len(mnist_train) // batch_size   # 60000 / 100 = 600
    
    for i, (batch_images, batch_labels) in enumerate(data_loader):
        
        X = Variable(batch_images.view(-1, 28 * 28))
        Y = Variable(batch_labels)
        
        pred = model(X)
        cost = loss(pred, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if (i+1) % total_batch == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'%(epoch+1, num_epochs, i+1, total_batch, cost.data[0]))
    
print("Learning Finished!")

Epoch [1/10], lter [600/600] Loss: 0.1375
Epoch [2/10], lter [600/600] Loss: 0.1774
Epoch [3/10], lter [600/600] Loss: 0.0568
Epoch [4/10], lter [600/600] Loss: 0.0231
Epoch [5/10], lter [600/600] Loss: 0.0223
Epoch [6/10], lter [600/600] Loss: 0.0034
Epoch [7/10], lter [600/600] Loss: 0.0314
Epoch [8/10], lter [600/600] Loss: 0.0410
Epoch [9/10], lter [600/600] Loss: 0.0013
Epoch [10/10], lter [600/600] Loss: 0.0107
Learning Finished!


### Test

In [7]:
model.eval()

correct = 0
total = 0

for images, labels in mnist_test:
    
    X = Variable(images.view(-1, 28 * 28))
    outputs = model(X)
    
    _, pred = torch.max(outputs.data, 1)  # arg max
    
    total += 1
    correct += (pred == labels).sum()
    
print('Accuracy of test images: %f %%' % (100 * correct / total))
print(correct, total)

Accuracy of test images: 97.270000 %
9727 10000


### Loss Function

* L1 loss : Mostly used as a regularizer. 
* MSE loss : Used as loss function for regression problems.
* Cross-entropy loss : Used for binary and multi-class classification problems.
* NLL Loss : Used for classification problems and allows us to use specific weights to handle imbalanced datasets.
* NLL Loss2d : Used for pixel-wise classification, mostly for problems related to image segmentation.

## 2. Cats and Dogs Classification